In [1]:
import pandas as pd
import numpy as np
import os
import re

# # Set working directory
# if not "/data/tables" in os.getcwd():
#     os.chdir("../data/tables")

from pyspark.sql import SparkSession
from pyspark.shell import spark
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
import matplotlib.pyplot as plt

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/09/12 12:56:58 WARN Utils: Your hostname, Kes-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.243.118 instead (on interface en0)
22/09/12 12:56:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/12 12:56:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/12 12:56:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/12 12:56:59 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/

Using Python version 3.9.7 (default, Sep 16 2021 08:50:36)
Spark context Web UI available at http://10.13.243.118:4042
Spark context available as 'sc' (master = local[*], app id = local-1662951419283).
SparkSession available as 'spark'.


In [2]:
sdf = spark.read.parquet("../data/tables/transactions_20220228_20220828_snapshot")

In [5]:
sdf.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



In [7]:
consumer_fraud_df = pd.read_csv("../data/tables/consumer_fraud_probability.csv")
merchant_fraud_df = pd.read_csv("../data/tables/merchant_fraud_probability.csv")

In [11]:
consumer_fraud_df.head(10)

,user_id,order_datetime,fraud_probability
0,6228,2021-12-19,97.629808
1,21419,2021-12-10,99.247380
2,5606,2021-10-17,84.058250
3,3101,2021-04-17,91.421921
4,22239,2021-10-19,94.703425
5,16556,2022-02-20,89.656633
6,10278,2021-09-28,83.591367
7,15790,2021-12-30,71.770659
8,5233,2021-08-29,85.871233
9,230,2021-08-28,86.283288


In [12]:
merchant_fraud_df.head(20)

,merchant_abn,order_datetime,fraud_probability
0,19492220327,2021-11-28,44.403659
1,31334588839,2021-10-02,42.755301
2,19492220327,2021-12-22,38.867790
3,82999039227,2021-12-19,94.134700
4,90918180829,2021-09-02,43.325517
5,31334588839,2021-12-26,38.361660
6,23686790459,2021-12-10,79.454344
7,14827550074,2021-11-26,46.457756
8,31334588839,2021-11-26,36.209713
9,19492220327,2021-12-18,33.819672


## Merchant Fraud
Check if there is missing data

In [47]:
merchant_sdf = spark.read.parquet("../data/tables/tbl_merchants.parquet")
merchant_sdf.printSchema()
merchant_df = merchant_sdf.toPandas()

root
 |-- name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- merchant_abn: long (nullable = true)



In [54]:
#merchant_sdf.select("merchant_abn").distinct().count()
# 4026 entries distince values of merchant

fraud_union = set(merchant_fraud_df["merchant_abn"].unique()) # 61? unique cases
all_union = set(merchant_df["merchant_abn"].unique())
len(all_union.intersection(fraud_union))



48

Only 48/4026 merchants have fraud data
13/61 of the fraud data cannot be matched with any merchant abn

In [41]:
merchant_df = merchant_sdf.toPandas()
merge_df = merchant_fraud_df.merge(merchant_df,on="merchant_abn")["merchant_abn"].unique()


48